In [15]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os

# Import geomap of the Netherlands
mapdf = gpd.read_file('Data/NL-geomap/gemeente_2018_v3.shp')
mapdf = mapdf[mapdf["H2O"] == "NEE"]
mapdf.sort_values("GM_NAAM", inplace=True)
mapdf = mapdf.reset_index(drop=True)
print(mapdf.columns)

# Compute distances between all municipalities
mapdf['GEO_CENTROID'] = mapdf.centroid
centres = mapdf['GEO_CENTROID']
n_mun = mapdf.shape[0]

dist_matrix = np.zeros(shape=(n_mun, n_mun))
for i in range(0, n_mun):
    dist_matrix[i] = centres.distance(centres[i])

# Compute mobility based gravity model, without distances
population = mapdf['AANT_INW']
mob_matrix = np.zeros(shape=(n_mun, n_mun))
for i in range(0, n_mun):
    for j in range(0, n_mun):
        if i != j:
            mob_matrix[i,j] = population[i] * population[j]

# Import original data to normalize to
df_mobmat = pd.read_csv('Data/Mob_grav_freq.csv', delimiter=',')
mobmat = df_mobmat[df_mobmat.columns[1:]].to_numpy()

# Normalize row sums to original data, so total amount traveled per municipality is the same.
for i in range(n):
    diff = np.nansum(mob_matrix[i]) / np.nansum(mobmat[i])
    mob_matrix[i] = mob_matrix[i] / diff

# Save the new matrix
path = os.getcwd() + '/Data/'
np.save(path + 'Randomized_mob_grav', mob_matrix)


Index(['GM_CODE', 'GM_NAAM', 'H2O', 'OAD', 'STED', 'BEV_DICHTH', 'AANT_INW',
       'AANT_MAN', 'AANT_VROUW', 'P_00_14_JR',
       ...
       'AV20PODIUM', 'AF_MUSEUM', 'AV5_MUSEUM', 'AV10MUSEUM', 'AV20MUSEUM',
       'JRSTATCODE', 'JAAR', 'Shape_Leng', 'Shape_Area', 'geometry'],
      dtype='object', length=205)


In [9]:
import os


folder_to_clear = "D:/University/thesis/covid-gravity-model/Data/Model_V1/Data/High/Seed_5"

for demo in range(0, 11):
    path = folder_to_clear + '/Runs_ref_3_3_5_' + str(demo)
    for run in range(0, 10):
        os.remove(path + '/Contacts_per_agent_' + str(run) + '.npy')
